In [ ]:
import influxdb_client
import pandas as pd

In [ ]:
TOKEN = '80pzM9UJharBRVfSbBIfecydGJsX80ihx2IZsd7UmAZUhUumZTIiX9jUqQNytXPxeVuZlgLmHAY6JOyHnLZuJw=='
URL = 'http://localhost:8086'
ORG = 'VCT'
BUCKET = 'HOUSE_HOLD_ENERGY'

In [ ]:
client = influxdb_client.InfluxDBClient(url=URL, token=TOKEN, org=ORG,  timeout=30000)

In [ ]:
query_api = client.query_api()

In [ ]:
query = f'''
from(bucket: "{BUCKET}")
  |> range(start: 2007-01-01T00:00:00Z, stop: 2007-12-31T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "house_energy")
  |> filter(fn: (r) => r["_field"] == "Global_active_power" or 
                      r["_field"] == "Global_reactive_power" or 
                      r["_field"] == "Voltage" or 
                      r["_field"] == "Global_intensity" or 
                      r["_field"] == "Sub_metering_1" or 
                      r["_field"] == "Sub_metering_2" or 
                      r["_field"] == "Sub_metering_3")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

In [ ]:
data = query_api.query_data_frame(query=query)

In [ ]:
data

In [ ]:
data.reset_index(inplace=True, drop=True)
data.info()

#### Limpando dataframe

##### Retirando dados desnecessários

In [ ]:
data.drop(columns=['result', 'table', '_start', '_stop', '_measurement'], inplace=True)
data

In [ ]:
data.info()

In [ ]:
columns_float = list(data.iloc[:, 1:].columns)
columns_float

#### Removendo dados nulos

In [ ]:
rows_to_delete = list(data.query('Global_active_power == "?"').index)

In [ ]:
data.drop(index=rows_to_delete)

In [ ]:
data = data.drop(index=rows_to_delete)

#### Convertendo colunas numericas

In [ ]:
data[columns_float] = data[columns_float].astype(float)

In [ ]:
data.info()

In [ ]:
data.plot(kind='line', y='Global_active_power', figsize=(20, 10))

#### Agregando dados por semana

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.set_index('_time', inplace=True, drop=True)

In [ ]:
data = data.resample('W').sum()

In [ ]:
data.head()

In [ ]:
data.drop(columns='index', inplace=True)
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

###### 229 / 5.000
(global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) representa a energia ativa consumida a cada minuto (em watts-hora) na residência por equipamentos elétricos não medidos nas submedições 1, 2 e 3.

In [ ]:
data['energy_consumed'] = (data['Global_active_power'] * 1000 / 60) - data['Sub_metering_1'] - data['Sub_metering_2'] - data['Sub_metering_3']
data['energy_consumed']

In [ ]:
data.head()

In [ ]:
import math

In [ ]:
for i in data.columns:
    data[i] = data[i].round()

In [ ]:
data

In [ ]:
data.describe()

#### Media das medicoes

In [ ]:
estatisticas  = ['media', 'max', 'min', 'std', 'count']